In [16]:


import os
# import torch
import numpy as np
import pandas as pd

from tqdm import tqdm
# from scipy import sparse
# from datetime import datetime
# from scipy.linalg import solve
# from scipy.optimize import minimize
# from scipy.sparse.linalg import spsolve
from bayes_opt import BayesianOptimization
from sklearn.preprocessing import MinMaxScaler
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import cross_val_predict



In [3]:


DATA_PATH = '../data/testing/ncaam_sample_data.csv'
def load_data(data_path):
    return pd.read_csv(data_path)

m_data = load_data(DATA_PATH)
scaler = MinMaxScaler()
m_data['team_score']= m_data['team_score'].clip(36, 120)
m_data['continuous_target'] = scaler.fit_transform(m_data['team_score'].values.reshape(-1, 1))
m_data['continuous_target_2'] = m_data['team_fgm'].copy()/m_data['team_fga'].copy()
m_data['date'] = pd.to_datetime(m_data['date'])
m_data = m_data.sort_values(by=['date', 'team_name']).reset_index(drop=True)



In [31]:
class Optimizer:
    def __init__(self):
        pass

    def load_data(self):
        raise NotImplementedError("Subclasses must implement the load_data method.")
    
    def optimize(self):
        raise NotImplementedError("Subclasses must implement the optimize method.")
    

class EloOptimizer(Optimizer):
    def __init__(self, protag_col='team', antag_col='opponent', stat_cols=['team_sq_score'], order_col='date', meta_cols=['location']):
        super().__init__()
        self.protag_col = protag_col
        self.antag_col = antag_col
        self.stat_cols = stat_cols
        self.order_col = order_col  
        self.meta_cols = meta_cols
        

    def load_data(self, data_or_path):
        if type(data_or_path) == str:
            self.data = pd.read_csv(data_or_path)
        else:
            self.data = data_or_path

        for col in [self.protag_col, self.antag_col]+self.stat_cols+[self.order_col]+self.meta_cols:
            assert(col in self.data.columns), f"{col} not found in data columns."
        self.preprocess_data()

    def preprocess_data(self):

        if len(self.stat_cols) > 1:
            self.data = self.data.melt(id_vars=[self.order_col, self.protag_col, self.antag_col]+self.meta_cols, value_vars=self.stat_cols, var_name='stat', value_name='stat_value')
        
        self.data = self.data.sort_values(by=self.order_col)
        return self.data
    
    def checks(self, k, meta_k, priors, initial_rating):
        assert(type(priors) == dict), "Priors must be a dictionary with key as protag and value as Elo rating."
        if k is None:
            raise ValueError("Must provide K value")
        else:
            assert(type(k) in [dict, float, int]), "K must be a single numeric quantity, or a dict that has unique values for each stat"
        assert(type(initial_rating) in [dict, float, int]), "Initial rating must be a single numeric quantity, or a dict that has unique values for each stat"
        if len(meta_k) > 0:
            assert(type(meta_k) == dict), "Meta K must be a dictionary with key as meta column and value as K value"
            for meta_col, k_val in meta_k.items():
                assert(type(k_val) in [dict, float, int]), "K must be a single numeric quantity, or a dict that has unique values for each stat"

    def optimize(self):

        ## taking a shortcut for now
        def opt_helper(k, is_home_continuous_target, is_home_continuous_target_2):
            meta_k = {'is_home': {'continuous_target': is_home_continuous_target, 'continuous_target_2': is_home_continuous_target_2}}
            k = int(k)
            rtgs = self.run_history(k=k, meta_adj=meta_k)
            ## log loss formula, negative because we are maximizing
            log_loss = np.mean(rtgs['protag_result'] * np.log(rtgs['protag_win_prob']) + (1 - rtgs['protag_result']) * np.log(1 - rtgs['protag_win_prob']))
            return log_loss

        pbounds = {
            'k': (10, 75),
            'is_home_continuous_target': (0, 100),
            'is_home_continuous_target_2': (0, 100)
        }

        optimizer = BayesianOptimization(
            f=opt_helper,
            pbounds=pbounds,
            random_state=17,
        )

        optimizer.maximize(
            init_points=10,
            n_iter=35,
        )
        print(optimizer.max['target'], optimizer.max['params'])
        return optimizer.max['params']

    def run_history(self, k, meta_adj={}, priors={}, initial_rating=1500):

        self.checks(k, meta_adj, priors, initial_rating)

        protags = self.data[self.protag_col].unique()
        antags = self.data[self.antag_col].unique()
        protags = sorted(protags)
        antags = sorted(antags)
        stats = sorted(self.stat_cols)

        protag_idx_map = {protag: idx for idx, protag in enumerate(protags)}
        antag_idx_map = {antag: idx + len(protags) for idx, antag in enumerate(antags)}
        stat_idx_map = {stat: idx for idx, stat in enumerate(stats)}

        self.data['protag_idx'] = self.data[self.protag_col].map(protag_idx_map)
        self.data['antag_idx'] = self.data[self.antag_col].map(antag_idx_map) 
        if len(stats) > 1:
            self.data['stat_idx'] = self.data['stat'].map(stat_idx_map)
        else:
            self.data['stat'] = self.stat_cols[0]
            self.data['stat_idx'] = 0
        
        if len(stats) == 1:
            self.data['stat_value'] = self.data[self.stat_cols[0]]

        i = 0
        for meta_col in self.meta_cols:
            i += 1
            self.data[f'meta_adj_{i}'] = self.data[[meta_col, 'stat']].apply(lambda x: meta_adj[meta_col][x['stat']], axis=1)
            self.data[f'meta_adj_{i}'] = self.data[f'meta_adj_{i}']*self.data[meta_col]
            
        self.data['meta_adj'] = self.data[[f'meta_adj_{i}' for i in range(1, i+1)]].sum(axis=1)

        ratings_mat = np.zeros((len(protags)+len(antags),len(self.stat_cols)))

        if len(stats)==1:
            for entity, prior in priors.items():
                if entity in protags:
                    ratings_mat[protag_idx_map[entity], 0] = prior
                else:
                    ratings_mat[antag_idx_map[entity], 0] = prior
        else:
            for entity, prior in priors.items():
                if entity in protags:
                    for idx, stat in enumerate(stats):
                        ratings_mat[protag_idx_map[entity], idx] = prior[stat]
                else:
                    for idx, stat in enumerate(stats):
                        ratings_mat[antag_idx_map[entity], idx] = prior[stat]

        no_prior_protags = set(protags) - set(priors.keys())
        no_prior_antags = set(antags) - set(priors.keys())
        for protag in no_prior_protags:
            if len(stats)==1:
                ratings_mat[protag_idx_map[protag], 0] = initial_rating
            else:
                for idx, stat in enumerate(stats):
                    ratings_mat[protag_idx_map[protag], idx] = initial_rating
        for antag in no_prior_antags:
            if len(stats)==1:
                ratings_mat[antag_idx_map[antag], 0] = initial_rating
            else:
                for idx, stat in enumerate(stats):
                    ratings_mat[antag_idx_map[antag], idx] = initial_rating

        ratings = []
        for rating_period, results in tqdm(self.data.groupby(self.order_col), total=self.data[self.order_col].nunique()):
            
            protag_idxs = results['protag_idx'].values
            antag_idxs = results['antag_idx'].values
            stat_idxs = results['stat_idx'].values

            protag_ratings = ratings_mat[protag_idxs, stat_idxs]
            antag_ratings = ratings_mat[antag_idxs, stat_idxs]

            if len(meta_adj) > 0:
                meta_values = results['meta_adj'].values
                protag_ratings += meta_values

            protag_win_probs = 1 / (1 + 10**((antag_ratings - protag_ratings) / 400))
            antag_win_probs = 1 - protag_win_probs

            ## todo: upgrade for multiple stats
            protag_wins = results['stat_value'].values
            antag_wins = 1 - protag_wins

            protag_update = k * (protag_wins - protag_win_probs)
            antag_update = k * (antag_wins - antag_win_probs)

            ratings_mat[protag_idxs, stat_idxs] += protag_update
            ratings_mat[antag_idxs, stat_idxs] += antag_update

            if len(meta_adj) > 0:
                protag_ratings -= meta_values

            to_append = pd.DataFrame({
                self.order_col: rating_period,
                'protag': results[self.protag_col],
                'antag': results[self.antag_col],
                'stat': results['stat'],    
                'protag_rating': protag_ratings,
                'antag_rating': antag_ratings,
                'protag_win_prob': protag_win_probs,
                'antag_win_prob': antag_win_probs,
                'protag_update': protag_update,
                'antag_update': antag_update,
                'protag_result': protag_wins,
                'antag_result': antag_wins,
                'protag_postgame_rating': protag_ratings+protag_update,
                'antag_postgame_rating': antag_ratings+antag_update,
            })

            ratings.append(to_append)

        ratings = pd.concat(ratings).reset_index(drop=True)

        return ratings
    

elo = EloOptimizer(protag_col='team_name', antag_col='opp_name', stat_cols=['continuous_target','continuous_target_2'], order_col='date', meta_cols=['is_home'])
elo.load_data(m_data)
rtgs = elo.run_history(k=45, initial_rating=1500, meta_adj={'is_home':{
    'continuous_target': 40,
    'continuous_target_2': 25
}})
rtgs



100%|██████████| 2674/2674 [00:00<00:00, 2883.08it/s]


,date,protag,antag,stat,protag_rating,antag_rating,protag_win_prob,antag_win_prob,protag_update,antag_update,protag_result,antag_result,protag_postgame_rating,antag_postgame_rating
0,2002-11-14,Alabama,Oklahoma,continuous_target,1500.000000,1500.000000,0.500000,0.500000,-5.357143,5.357143,0.380952,0.619048,1494.642857,1505.357143
1,2002-11-14,Syracuse,Memphis,continuous_target_2,1500.000000,1500.000000,0.500000,0.500000,-6.380597,6.380597,0.358209,0.641791,1493.619403,1506.380597
2,2002-11-14,Oklahoma,Alabama,continuous_target_2,1500.000000,1500.000000,0.500000,0.500000,-3.820755,3.820755,0.415094,0.584906,1496.179245,1503.820755
3,2002-11-14,Alabama,Oklahoma,continuous_target_2,1500.000000,1500.000000,0.500000,0.500000,-1.551724,1.551724,0.465517,0.534483,1498.448276,1501.551724
4,2002-11-14,Memphis,Syracuse,continuous_target_2,1500.000000,1500.000000,0.500000,0.500000,-3.629032,3.629032,0.419355,0.580645,1496.370968,1503.629032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413115,2022-04-02,Kansas,Villanova,continuous_target_2,1503.434167,1557.041366,0.423459,0.576541,5.110991,-5.110991,0.537037,0.462963,1508.545158,1551.930375
413116,2022-04-04,North Carolina,Kansas,continuous_target,1572.190795,1592.941933,0.470172,0.529828,-3.479176,3.479176,0.392857,0.607143,1568.711619,1596.421109
413117,2022-04-04,Kansas,North Carolina,continuous_target,1568.666758,1563.774036,0.507041,0.492959,-3.531118,3.531118,0.428571,0.571429,1565.135639,1567.305154
413118,2022-04-04,Kansas,North Carolina,continuous_target_2,1508.545158,1548.721302,0.442438,0.557562,-0.136993,0.136993,0.439394,0.560606,1508.408165,1548.858294


In [32]:
elo.optimize()

|   iter    |  target   | is_hom... | is_hom... |     k     |
-------------------------------------------------------------


100%|██████████| 2674/2674 [00:00<00:00, 3431.54it/s]


| 1         | -0.676    | 29.47     | 53.06     | 22.45     |


100%|██████████| 2674/2674 [00:00<00:00, 3009.64it/s]


| 2         | -0.6813   | 6.79      | 78.7      | 52.66     |


...

100%|██████████| 2674/2674 [00:00<00:00, 3416.23it/s]


| 44        | -0.6712   | 15.3      | 5.335     | 45.31     |


100%|██████████| 2674/2674 [00:00<00:00, 3445.31it/s]


| 45        | -0.6712   | 15.37     | 4.012     | 47.35     |
-0.6712318829093019 {'is_home_continuous_target': 15.348983572742537, 'is_home_continuous_target_2': 4.872094964602402, 'k': 48.63908768703055}


{'is_home_continuous_target': 15.348983572742537,
 'is_home_continuous_target_2': 4.872094964602402,
 'k': 48.63908768703055}

KeyError: 'stat'

In [6]:

last_rating =rtgs.drop_duplicates(subset=['protag','stat'], keep='last')[['date','protag','stat','protag_postgame_rating']].copy()
last_rating.sort_values(by=['protag_postgame_rating'], ascending=[False], inplace=True)
last_rating

NameError: name 'rtgs' is not defined

In [42]:
test2 = last_rating.pivot(index=['date','protag'], columns=['stat'], values=['protag_postgame_rating']).reset_index()
test2.columns=['date','protag','ppg','fg%']

In [43]:
test2.corr()

C:\Users\Blake\AppData\Local\Temp\ipykernel_35324\1000051891.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  test2.corr()


,ppg,fg%
ppg,1.000000,0.818036
fg%,0.818036,1.000000


In [34]:
m_data.melt(id_vars=['date', 'team_name', 'opp_name', 'is_home'], value_vars=['continuous_target', 'continuous_target_2'], var_name='stat', value_name='stat_value')

,date,team_name,opp_name,is_home,stat,stat_value
0,2002-11-14,Alabama,Oklahoma,0,continuous_target,0.380952
1,2002-11-14,Memphis,Syracuse,0,continuous_target,0.404762
2,2002-11-14,Oklahoma,Alabama,0,continuous_target,0.309524
3,2002-11-14,Syracuse,Memphis,0,continuous_target,0.321429
4,2002-11-15,E Washington,Wisconsin,-1,continuous_target,0.226190
...,...,...,...,...,...,...
413115,2022-04-02,Kansas,Villanova,0,continuous_target_2,0.537037
413116,2022-04-02,North Carolina,Duke,0,continuous_target_2,0.421875
413117,2022-04-02,Villanova,Kansas,0,continuous_target_2,0.385965
413118,2022-04-04,Kansas,North Carolina,0,continuous_target_2,0.439394


In [33]:
m_data

,season,team_score,opp_score,is_home,numot,team_fgm,team_fga,team_fgm3,team_fga3,team_ftm,...,opp_ast,opp_to,opp_stl,opp_blk,opp_pf,team_name,opp_name,date,continuous_target,continuous_target_2
0,2003,68,62,0,0,27,58,3,14,11,...,8,18,9,2,20,Alabama,Oklahoma,2002-11-14,0.380952,0.465517
1,2003,70,63,0,0,26,62,8,20,10,...,7,12,8,6,16,Memphis,Syracuse,2002-11-14,0.404762,0.419355
2,2003,62,68,0,0,22,53,2,10,16,...,13,23,7,1,22,Oklahoma,Alabama,2002-11-14,0.309524,0.415094
3,2003,63,70,0,0,24,67,6,24,9,...,16,13,4,4,18,Syracuse,Memphis,2002-11-14,0.321429,0.358209
4,2003,55,81,-1,0,20,46,3,11,12,...,12,9,9,3,18,E Washington,Wisconsin,2002-11-15,0.226190,0.434783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206555,2022,81,65,0,0,29,54,13,24,10,...,12,9,3,0,11,Kansas,Villanova,2022-04-02,0.535714,0.537037
206556,2022,81,77,0,0,27,64,10,26,17,...,12,4,7,4,18,North Carolina,Duke,2022-04-02,0.535714,0.421875
206557,2022,65,81,0,0,22,57,13,31,8,...,18,7,4,4,8,Villanova,Kansas,2022-04-02,0.345238,0.385965
206558,2022,72,69,0,0,29,66,6,17,8,...,9,13,2,6,13,Kansas,North Carolina,2022-04-04,0.428571,0.439394


149